In [98]:
# sample of training the titanic data sets and creating a submission using AutoML 
# for the popular Kaggle Titanic challenge. 
# Note: feature engineering approach copied from: 
# https://medium.com/i-like-big-data-and-i-cannot-lie/how-i-scored-in-the-top-9-of-kaggles-titanic-machine-learning-challenge-243b5f45c8e9 
# algorithn used in the above link was RandomForestClassifier. Automated ML will choose the best classifier!


In [99]:
# Uncomment below to Install azureml.core in the current Jupyter kernel
# import sys
# !{sys.executable} -m  pip install azureml.core --user

In [100]:
# Uncomment below to Install azureml.train.automl in the current Jupyter kernel
# import sys
# !{sys.executable} -m  pip install azureml.train.automl --user

In [101]:
subscription_id = "c8a23972-1b42-43fa-9bda-92e665014f30"
resource_group = "auto-ml-local-fe"
workspace_name = "kappgle-ml-local-fe"
workspace_region = "eastus2"

In [102]:
# Import the Workspace class and check the Azure ML SDK version.
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      exist_ok=True)
ws.get_details()

{'applicationInsights': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourcegroups/auto-ml-local-fe/providers/microsoft.insights/components/kappgleminsightswmnmvabj',
 'containerRegistry': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourcegroups/auto-ml-local-fe/providers/microsoft.containerregistry/registries/kappglemacrkkiqdpqv',
 'creationTime': '2019-02-12T13:54:21.7891262+00:00',
 'description': '',
 'friendlyName': 'kappgle-ml-local-fe',
 'id': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourceGroups/auto-ml-local-fe/providers/Microsoft.MachineLearningServices/workspaces/kappgle-ml-local-fe',
 'identityPrincipalId': '558fc346-bace-4ffd-a7ee-e45bb4770bfe',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityType': 'SystemAssigned',
 'keyVault': '/subscriptions/c8a23972-1b42-43fa-9bda-92e665014f30/resourcegroups/auto-ml-local-fe/providers/microsoft.keyvault/vaults/kappglemkeyvaultnlazmtvt',
 'location': 'eastus2',
 'name': 'kappgle-

In [103]:
import azureml.core
import pandas as pd
from azureml.core.workspace import Workspace
import logging
import os

# ws = Workspace.from_config()
# choose a name for the run history container in the workspace
experiment_name = 'automated-ml-classification'
# project folder
project_folder = './automated-ml-classification'

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

,
Location,eastus2
Project Directory,./automated-ml-classification
Resource Group,auto-ml-local-fe
SDK version,1.0.10
Subscription ID,c8a23972-1b42-43fa-9bda-92e665014f30
Workspace,kappgle-ml-local-fe


In [104]:
import pickle
import sys
import os
import pandas
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve

#get the titanic training data set
url = "https://raw.githubusercontent.com/TheovanKraay/mldata/master/kaggle-train.csv"
titanic = pandas.read_csv(url)

In [105]:
# wrangling and feature engineering 
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}
titanic['Title'] = titanic.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
# map the normalized titles to the current titles 
titanic.Title = titanic.Title.map(normalized_titles)

# group by Sex, Pclass, and Title 
grouped = titanic.groupby(['Sex','Pclass', 'Title'])  
# view the median Age by the grouped features 
grouped.Age.median()
# apply the grouped median value on the Age NaN
titanic.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))

# fill Cabin NaN with U for unknown
titanic.Cabin = titanic.Cabin.fillna('U')
# find most frequent Embarked value and store in variable
most_embarked = titanic.Embarked.value_counts().index[0]

# fill NaN with most_embarked value
titanic.Embarked = titanic.Embarked.fillna(most_embarked)
# fill NaN with median fare
titanic.Fare = titanic.Fare.fillna(titanic.Fare.median())

# size of families (including the passenger)
titanic['FamilySize'] = titanic.Parch + titanic.SibSp + 1

# map first letter of cabin to itself
titanic.Cabin = titanic.Cabin.map(lambda x: x[0])

print(titanic)

     PassengerId  Survived  Pclass  \
0    1            0         3        
1    2            1         1        
2    3            1         3        
3    4            1         1        
4    5            0         3        
5    6            0         3        
6    7            0         1        
7    8            0         3        
8    9            1         3        
9    10           1         2        
10   11           1         3        
11   12           1         1        
12   13           0         3        
13   14           0         3        
14   15           0         3        
15   16           1         2        
16   17           0         3        
17   18           1         2        
18   19           0         3        
19   20           1         3        
20   21           0         2        
21   22           1         2        
22   23           1         3        
23   24           1         1        
24   25           0         3        
25   26     

In [106]:
# Convert the male and female groups to integer form
titanic.Sex = titanic.Sex.map({"male": 0, "female":1})
# create dummy variables for categorical features
pclass_dummies = pd.get_dummies(titanic.Pclass, prefix="Pclass")
title_dummies = pd.get_dummies(titanic.Title, prefix="Title")
cabin_dummies = pd.get_dummies(titanic.Cabin, prefix="Cabin")
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix="Embarked")
# concatenate dummy columns with main dataset
titanic_dummies = pd.concat([titanic, pclass_dummies, title_dummies, cabin_dummies, embarked_dummies], axis=1)

# drop categorical fields
titanic_dummies.drop(['Pclass', 'Title', 'Cabin', 'Embarked', 'Name', 'Ticket'], axis=1, inplace=True)

titanic_dummies.head()

,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,FamilySize,Pclass_1,Pclass_2,...,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S
0,1,0,0,22.0,1,0,7.2500,2,0,0,...,0,0,0,0,0,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,2,1,0,...,1,0,0,0,0,0,0,1,0,0
2,3,1,1,26.0,0,0,7.9250,1,0,0,...,0,0,0,0,0,0,1,0,0,1
3,4,1,1,35.0,1,0,53.1000,2,1,0,...,1,0,0,0,0,0,0,0,0,1
4,5,0,0,35.0,0,0,8.0500,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [107]:


#dflow_X = titanic_dummies[['Sex', 'Age', 'SibSp', 'Parch', 'Cabin', 'Embarked', 'Fare' ,'Title', 'FamilySize']]
dflow_X = titanic_dummies[['Sex', 'Age', 'SibSp', 'Parch', 'Fare','FamilySize',  'Pclass_1', 'Pclass_2', 'Pclass_3', 'Title_Master','Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer','Title_Royalty', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D','Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_U', 'Embarked_C','Embarked_Q', 'Embarked_S']]
dflow_y = titanic_dummies['Survived']

In [108]:
from sklearn.model_selection import train_test_split

#split the dataset
x_df = dflow_X
y_df = dflow_y
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=223)

In [109]:
#configure AutoML settings
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 30,
    "primary_metric" : 'accuracy',
    "preprocess" : True,
    "verbosity" : logging.INFO,
    "n_cross_validations": 5
}

In [ ]:
from azureml.train.automl import AutoMLConfig

# local compute - set the parameters
automated_ml_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automated_ml_errors.log',
                             path = project_folder,
                             X = x_train.values,
                             y = y_train.values.flatten(),
                             **automl_settings)

In [ ]:
#run the training

from azureml.core.experiment import Experiment
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(automated_ml_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_f1c39107-ed81-4cde-829b-6fb1a216d115
*******************************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
TRAINFRAC: Fraction of the training data to train on.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
*******************************************************************************************************************

 ITERATION   PIPELINE                                       TRAINFRAC  DURATION      METRIC      BEST
         0   RobustScaler LogisticRegression                1.0000     0:00:16       0.8175    0.8175
         1   StandardScalerWrapper KNN                      1.0000     0:00:22       0.7921    0.8175
         2   MaxAbsScaler LogisticRegression                1.0000

In [ ]:
#get the best fitted model
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
#predict against the Kaggle provided test data for the Titanic
url = "https://raw.githubusercontent.com/TheovanKraay/mldata/master/kaggle-test.csv"
test = pandas.read_csv(url)

#***Wrangle and Feature engineer the test data to be predicted against
test['Title'] = test.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
# map the normalized titles to the current titles 
test.Title = test.Title.map(normalized_titles)
# group by Sex, Pclass, and Title 
grouped = test.groupby(['Sex','Pclass', 'Title'])  
# view the median Age by the grouped features 
grouped.Age.median()
# apply the grouped median value on the Age NaN
test.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))

# fill Cabin NaN with U for unknown
test.Cabin = test.Cabin.fillna('U')
# find most frequent Embarked value and store in variable
most_embarked = test.Embarked.value_counts().index[0]

# fill NaN with most_embarked value
test.Embarked = test.Embarked.fillna(most_embarked)
# fill NaN with median fare
test.Fare = test.Fare.fillna(test.Fare.median())

# size of families (including the passenger)
test['FamilySize'] = test.Parch + test.SibSp + 1

# map first letter of cabin to itself
test.Cabin = test.Cabin.map(lambda x: x[0])



In [ ]:
# Convert the male and female groups to integer form
test.Sex = test.Sex.map({"male": 0, "female":1})
# create dummy variables for categorical features
pclass_dummies = pd.get_dummies(test.Pclass, prefix="Pclass")
title_dummies = pd.get_dummies(test.Title, prefix="Title")
cabin_dummies = pd.get_dummies(test.Cabin, prefix="Cabin")
embarked_dummies = pd.get_dummies(test.Embarked, prefix="Embarked")
# concatenate dummy columns with main dataset
test_dummies = pd.concat([test, pclass_dummies, title_dummies, cabin_dummies, embarked_dummies], axis=1)

# drop categorical fields
test_dummies.drop(['Pclass', 'Title', 'Cabin', 'Embarked', 'Name', 'Ticket'], axis=1, inplace=True)

test_dummies.head()

In [ ]:
test_dummies.columns.values

In [ ]:
testpred = test_dummies[['Sex', 'Age', 'SibSp', 'Parch', 'Fare','FamilySize',  'Pclass_1', 'Pclass_2', 'Pclass_3', 'Title_Master','Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer','Title_Royalty', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D','Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_U', 'Embarked_C','Embarked_Q', 'Embarked_S']]
y_predict = fitted_model.predict(testpred.values)

#testpred = test[['PassengerId','Sex', 'Age', 'SibSp', 'Parch', 'Cabin', 'Embarked', 'Fare', 'Title', 'FamilySize']]
testpred = test[['PassengerId']]


In [ ]:
#generate a submission file
sub = pandas.DataFrame()
sub['PassengerId'] = test['PassengerId']
sub['Survived'] = y_predict
print(sub)

In [ ]:
#write the submission file
sub.to_csv('submission.csv',index=False)